In this notebook, the progress of the Low-Rank Multiplication program is demonstrated. It involves two matrices, where SVD is performed on matrix A while matrix B remains unchanged. The percentage determines the number of vectors to retain from the sigma matrix, which are then used to multiply matrix A by matrix B.

In [1]:
import numpy as np

def LRM_percentage(matrix_A, matrix_B, percentage=100):
    # SVD decomposition of matrix A
    U, S, VT = np.linalg.svd(matrix_A, full_matrices=False)
    
    # Determine number of singular values to use of the S matrix
    num_singular_values = len(S)
    k = int(np.ceil(num_singular_values * (percentage / 100)))
    k = max(1, min(k, num_singular_values))  # ensure at least 1, at most all

    # Keep only top-k singular values
    S_k = S[:k]
    
    print(f"\nUsing top {k} singular values ({percentage:.1f}%) for LRM")
    print("Original Singular Values (S):", S)
    print("Top-k Singular Values (S_k):", S_k)

    # Step 1: Multiply V^T and B
    step1 = VT[:k, :] @ matrix_B  # only use top-k rows of VT

    # Step 2: Scale each row by Sigma[i,i]
    total = np.zeros_like(step1)
    for i in range(k):
        total[i, :] = S_k[i] * step1[i, :]

    # Step 3: Multiply by U
    final_result = U[:, :k] @ total  # only first k columns of U
    print("Final Result of LRM (approx):", final_result, sep="\n")
    
    return final_result

# Example matrices, random numbers between -10 and 10
matrix_A = np.random.randint(-10, 11, size=(3, 4))  
matrix_B = np.random.randint(-10, 11, size=(4, 3))

# Direct multiplication for comparison
direct_result = matrix_A @ matrix_B

# Test LRM with different percentages
percentages = [100, 61.7, 33.3]
for p in percentages:
    result = LRM_percentage(matrix_A, matrix_B, percentage=p)
    error_percentage = (np.linalg.norm(direct_result - result, 'fro') / np.linalg.norm(direct_result, 'fro')) * 100
    print(f"Percentage Error: {error_percentage:.6f}%")



Using top 3 singular values (100.0%) for LRM
Original Singular Values (S): [15.12497522  9.30512914  6.68204283]
Top-k Singular Values (S_k): [15.12497522  9.30512914  6.68204283]
Final Result of LRM (approx):
[[ -32.   38.  -31.]
 [ -61.    5. -106.]
 [  22.  -48. -144.]]
Percentage Error: 0.000000%

Using top 2 singular values (61.7%) for LRM
Original Singular Values (S): [15.12497522  9.30512914  6.68204283]
Top-k Singular Values (S_k): [15.12497522  9.30512914]
Final Result of LRM (approx):
[[ -44.02788059   23.99135296  -40.8238979 ]
 [ -51.82578387   15.68503754  -98.50686254]
 [  19.17917183  -51.28536569 -146.30394106]]
Percentage Error: 13.066433%

Using top 1 singular values (33.3%) for LRM
Original Singular Values (S): [15.12497522  9.30512914  6.68204283]
Top-k Singular Values (S_k): [15.12497522]
Final Result of LRM (approx):
[[  -6.47680336   -6.32147208  -45.58942592]
 [ -14.66468176  -14.312983   -103.22289964]
 [ -20.0773458   -19.5958367  -141.32197919]]
Percentage E